# Zero2API Client

## Overview


Our purpose here is to illustrate data access (query) against an openly published table: 26 baboons 
with position fixes (in meters) every two seconds for a day, approximately. We have two forms of 
access: A basic table interface and a 'derived product' interface. The latter calculates the relative
dispersion of the baboon troop. 


The basic table-based API is used first. We'll call it `baboon1`. It supports queries over the four
table columns: time range, x-coordinate, y-coordinate (so a spatial bounding box) and individual identifier. 
We ignore the bounding box for this demonstration.


The second ('derived product') API was built separately from the first. In fact it makes use of the first 
`baboon1` interface to get its data. This is what we call *API composition*: Rather than make the basic 
API more complicated -- for example as we imagine new interesting calculations -- we push or compartmentalize 
the new calculation into other APIs.


### First (basic) API examples

In [30]:
# libraries used for talking with the two APIs
import pandas as pd
import requests

In [34]:
# use the 'baboons1' interface to get 11 rows of data

indiv = 10              # there are at least 38 GPS collars; we have here data for only 26 of them 
table = 'false'         # tabular data flag false means 'send the result as JSON'
t0 = '9:00:00'          # Time range begins at 2 minutes 52 seconds; duration 32 seconds
t1 = '9:00:22'          #   but only 22 seconds of data are returned

baseURL = 'https://rlu4ch9a57.execute-api.us-west-2.amazonaws.com/default/baboon1'
url     = baseURL + '?indiv={}&table={}&t0={}&t1={}'.format(indiv,table,t0,t1)
a=pd.read_json(url)     # the url contains the details of the query; issued here
a                       # the response is a pandas DataFrame with 11 rows of data

https://rlu4ch9a57.execute-api.us-west-2.amazonaws.com/default/baboon1?indiv=10&table=false&t0=9:00:00&t1=9:00:22


,indiv,time,x,y
0,10,9:00:00,300.463922,-405.358183
1,10,9:00:02,299.873931,-405.413842
2,10,9:00:04,299.228279,-405.658740
3,10,9:00:06,298.849794,-406.204193
4,10,9:00:08,298.827531,-406.794174
5,10,9:00:10,298.916586,-407.206047
6,10,9:00:12,298.916586,-407.339628
7,10,9:00:14,298.894322,-407.350760
8,10,9:00:16,298.894322,-407.395286
9,10,9:00:18,298.827531,-407.617921


In [44]:
# This illustrates some pandas DataFrame attributes

print(type(a), '\n\n', a, '\n\n', a.index, '\n\n', a.columns, '\n\n', a['time'], '\n\n', a['x'][0:3])

<class 'pandas.core.frame.DataFrame'> 

     indiv     time           x           y
0      10  0:03:04 -778.263144  428.061810
1      10  0:03:06 -778.207484  428.095205
2      10  0:03:08 -777.884659  428.351235
3      10  0:03:10 -777.795603  428.395762
4      10  0:03:12 -777.328063  428.362367
5      10  0:03:14 -777.072029  428.262181
6      10  0:03:16 -776.059025  428.707449
7      10  0:03:18 -775.112812  429.008006
8      10  0:03:20 -774.500557  429.130454
9      10  0:03:22 -773.220387  429.798357
10     10  0:03:24 -773.220387  429.653645 

 RangeIndex(start=0, stop=11, step=1) 

 Index(['indiv', 'time', 'x', 'y'], dtype='object') 

 0     0:03:04
1     0:03:06
2     0:03:08
3     0:03:10
4     0:03:12
5     0:03:14
6     0:03:16
7     0:03:18
8     0:03:20
9     0:03:22
10    0:03:24
Name: time, dtype: object 

 0   -778.263144
1   -778.207484
2   -777.884659
Name: x, dtype: float64


In [45]:
# test cell: try various types of query including bounding box (not working yet)

indiv = 10
table = 'false'
t0 = '0:02:52'
t1 = '0:03:24'
x0 = -770.
x1 = -780.
y0 =  430.
y1 =  420.

# full scan
# url = baseURL + '?indiv={}&table={}&t0={}&t1={}&x0={}&x1={}&y0={}&y1={}'.format(indiv,table,t0,t1,x0,x1,y1,y0)

# scan sans indiv
# url = baseURL + '?table={}&t0={}&t1={}&x0={}&x1={}&y0={}&y1={}'.format(table,t0,t1,x0,x1,y1,y0)

# query indiv and time: Working
# url = baseURL + '?indiv={}&table={}&t0={}&t1={}'.format(indiv,table,t0,t1)

# url = baseURL + '?table={}&t0={}&t1={}'.format(table,t0,t1)
# url = baseURL + '?indiv={}&table={}'.format(indiv,table)

# a=pd.read_json(url)
# print(url, '\n\n', a)

### Second (moment of inertia) API example

In [46]:
# obtain one moment of inertia value for the entire troupe using one minute of data

urlcomp = 'https://ycg3xmhjwj.execute-api.us-west-2.amazonaws.com/default/babooncomposition'
t0 = '11:00:00'
t1 = '11:01:00'
url= urlcomp + '?moi=true&t0={}&t1={}'.format(t0,t1)
a = requests.get(url)
print(a.content.decode('utf-8'))

moi is 10.30867843517405 from 775 locations


In [47]:
# test a bad query (the times t0 and t1 are not usable)
#   this should produce 'zero' or some sort of fail signal

urlcomp = 'https://ycg3xmhjwj.execute-api.us-west-2.amazonaws.com/default/babooncomposition'
t0 = 'abcd'
t1 = 'efgh'
url= urlcomp + '?moi=true&t0={}&t1={}'.format(t0,t1)
a = requests.get(url)
print(a.content.decode('utf-8'))

moi is 0.0 from 0 locations


In [48]:
# Loop over one hour in three-minute intervals to produce 20 moments of inertia

urlcomp = 'https://ycg3xmhjwj.execute-api.us-west-2.amazonaws.com/default/babooncomposition'
b=[]
for i in range(20):
    mymin0 = i * 3
    mymin1 = mymin0 + 3
    if mymin0 < 10: mymin0string = '0' + str(mymin0)
    else:           mymin0string = str(mymin0)
    if mymin1 < 10: mymin1string = '0' + str(mymin1)
    else:           mymin1string = str(mymin1)
    t0 = '4:' + mymin0string + ':00'
    t1 = '4:' + mymin1string + ':00'
    if t1 == '4:60:00': t1 = '5:00:00'
    url= urlcomp + '?moi=true&t0={}&t1={}'.format(t0,t1)
    a = requests.get(url)
    b.append(float(a.content.decode("utf-8").split(' ')[2]))

print(b)

[47.24668210047696, 47.71861072775954, 48.92386733276165, 49.34812811013475, 48.8155080002324, 47.06645781119527, 49.71762649918903, 70.1851986629111, 84.94120700100285, 78.46260152031512, 66.99084135228593, 74.55004855452188, 86.66621643361388, 92.59470487945826, 90.57328670073204, 74.01396790676765, 55.93187417612399, 32.35153076898543, 20.589211909522685, 16.512057140423554]


In [32]:
# write as text file the accumulated moi values

with open('hour4_3min_moi_values.txt', 'w') as f:
    for item in b: f.write("%s\n" % item)

In [27]:
# obtain one moment of inertia value for the entire troupe using one minute of data
import requests
urlcomp = 'https://ycg3xmhjwj.execute-api.us-west-2.amazonaws.com/default/babooncomposition'
t0 = '9:00:00'       # notice these do not begin with '09' but rather just '9'
t1 = '9:05:00'
t2 = '9:05:00'
t3 = '9:10:00'

url= urlcomp + '?t0={}&t1={}&t2={}&t3={}'.format(t0,t1,t2,t3)
a = requests.get(url)
print(a.content.decode('utf-8'))

0.0,0.0,0,0,0.0


In [25]:
print(url)

https://ycg3xmhjwj.execute-api.us-west-2.amazonaws.com/default/babooncomposition?t0=09:00:00&t1=09:05:00&t2=09:05:00&t3=09:10:00


In [8]:
# obtain one moment of inertia value for the entire troupe using one minute of data
import requests
urlcomp = 'https://ycg3xmhjwj.execute-api.us-west-2.amazonaws.com/default/babooncomposition'
t0 = 'a'
t1 = 'b'
t2 = 'c'
t3 = 'd'
url= urlcomp + '?t0={}&t1={}&t2={}&t3={}'.format(t0,t1,t2,t3)
a = requests.get(url)
print(a.content.decode('utf-8'))

0.0,0.0,0,0,0.0


In [ ]:
# run the full day
import requests

def TimeBuilder():
    # Hour string begins '7', not '07' etc for single-digit hours
    t = []
    for i in range(24):
        for j in range(6):
            hrstring = str(i)
            jj = j * 10
            if jj == 0: minstring = '00'
            else:       minstring = str(jj)
            jjj = jj + 5
            if jjj == 5: minstring1 = '05'
            else:        minstring1 = str(jjj)
            jjjj = jjj + 5
            if jjjj < 60: 
                minstring2 = str(jjjj)
                t3 = hrstring + ':' + minstring2 + ':00'
            else: 
                t3 = hrstring + ':59:59'
            t0 = hrstring + ':' + minstring + ':00'
            t1 = hrstring + ':' + minstring1 + ':00'
            t2 = hrstring + ':' + minstring1 + ':00'
            # t3 = hrstring2 + ':' + minstring2 + ':00' bug where this fails to cross the hour boundary
            
            t.append((t0,t1,t2,t3))     # a tuple of four strings               
    return t

t = TimeBuilder()
urlcomp = 'https://ycg3xmhjwj.execute-api.us-west-2.amazonaws.com/default/babooncomposition'
myresults = []
for time in t:
    url= urlcomp + '?t0={}&t1={}&t2={}&t3={}'.format(time[0],time[1],time[2],time[3])
    print(url)
    a = requests.get(url)
    myresults.append(a.content.decode("utf-8").split(','))
    print('finished', time[0], time[1], time[2], time[3])
    
with open('moimoispeed_5min_1day.txt', 'w') as f:
    for item in myresults: f.write("%s\n" % item)
        
print(myresults)
    

https://ycg3xmhjwj.execute-api.us-west-2.amazonaws.com/default/babooncomposition?t0=0:00:00&t1=0:05:00&t2=0:05:00&t3=0:10:00
finished 0:00:00 0:05:00 0:05:00 0:10:00
https://ycg3xmhjwj.execute-api.us-west-2.amazonaws.com/default/babooncomposition?t0=0:10:00&t1=0:15:00&t2=0:15:00&t3=0:20:00
finished 0:10:00 0:15:00 0:15:00 0:20:00
https://ycg3xmhjwj.execute-api.us-west-2.amazonaws.com/default/babooncomposition?t0=0:20:00&t1=0:25:00&t2=0:25:00&t3=0:30:00
finished 0:20:00 0:25:00 0:25:00 0:30:00
https://ycg3xmhjwj.execute-api.us-west-2.amazonaws.com/default/babooncomposition?t0=0:30:00&t1=0:35:00&t2=0:35:00&t3=0:40:00
finished 0:30:00 0:35:00 0:35:00 0:40:00
https://ycg3xmhjwj.execute-api.us-west-2.amazonaws.com/default/babooncomposition?t0=0:40:00&t1=0:45:00&t2=0:45:00&t3=0:50:00
finished 0:40:00 0:45:00 0:45:00 0:50:00
https://ycg3xmhjwj.execute-api.us-west-2.amazonaws.com/default/babooncomposition?t0=0:50:00&t1=0:55:00&t2=0:55:00&t3=0:59:59
finished 0:50:00 0:55:00 0:55:00 0:59:59
http

finished 8:10:00 8:15:00 8:15:00 8:20:00
https://ycg3xmhjwj.execute-api.us-west-2.amazonaws.com/default/babooncomposition?t0=8:20:00&t1=8:25:00&t2=8:25:00&t3=8:30:00
finished 8:20:00 8:25:00 8:25:00 8:30:00
https://ycg3xmhjwj.execute-api.us-west-2.amazonaws.com/default/babooncomposition?t0=8:30:00&t1=8:35:00&t2=8:35:00&t3=8:40:00
finished 8:30:00 8:35:00 8:35:00 8:40:00
https://ycg3xmhjwj.execute-api.us-west-2.amazonaws.com/default/babooncomposition?t0=8:40:00&t1=8:45:00&t2=8:45:00&t3=8:50:00
finished 8:40:00 8:45:00 8:45:00 8:50:00
https://ycg3xmhjwj.execute-api.us-west-2.amazonaws.com/default/babooncomposition?t0=8:50:00&t1=8:55:00&t2=8:55:00&t3=8:59:59
finished 8:50:00 8:55:00 8:55:00 8:59:59
https://ycg3xmhjwj.execute-api.us-west-2.amazonaws.com/default/babooncomposition?t0=9:00:00&t1=9:05:00&t2=9:05:00&t3=9:10:00
finished 9:00:00 9:05:00 9:05:00 9:10:00
https://ycg3xmhjwj.execute-api.us-west-2.amazonaws.com/default/babooncomposition?t0=9:10:00&t1=9:15:00&t2=9:15:00&t3=9:20:00
fini

finished 16:10:00 16:15:00 16:15:00 16:20:00
https://ycg3xmhjwj.execute-api.us-west-2.amazonaws.com/default/babooncomposition?t0=16:20:00&t1=16:25:00&t2=16:25:00&t3=16:30:00
finished 16:20:00 16:25:00 16:25:00 16:30:00
https://ycg3xmhjwj.execute-api.us-west-2.amazonaws.com/default/babooncomposition?t0=16:30:00&t1=16:35:00&t2=16:35:00&t3=16:40:00
finished 16:30:00 16:35:00 16:35:00 16:40:00
https://ycg3xmhjwj.execute-api.us-west-2.amazonaws.com/default/babooncomposition?t0=16:40:00&t1=16:45:00&t2=16:45:00&t3=16:50:00
finished 16:40:00 16:45:00 16:45:00 16:50:00
https://ycg3xmhjwj.execute-api.us-west-2.amazonaws.com/default/babooncomposition?t0=16:50:00&t1=16:55:00&t2=16:55:00&t3=16:59:59
finished 16:50:00 16:55:00 16:55:00 16:59:59
https://ycg3xmhjwj.execute-api.us-west-2.amazonaws.com/default/babooncomposition?t0=17:00:00&t1=17:05:00&t2=17:05:00&t3=17:10:00
finished 17:00:00 17:05:00 17:05:00 17:10:00
https://ycg3xmhjwj.execute-api.us-west-2.amazonaws.com/default/babooncomposition?t0=1

In [ ]:
urlcomp = 'https://ycg3xmhjwj.execute-api.us-west-2.amazonaws.com/default/babooncomposition'
t0 = '13:00:00'
t1 = '13:05:00'
t2 = '13:05:00'
t3 = '13:10:00'
url= urlcomp + '?t0={}&t1={}&t2={}&t3={}'.format(t0,t1,t2,t3)
a = requests.get(url)
print(a.content.decode('utf-8'))